In [1]:
import requests
import pandas as pd
import datetime
import time

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

In [2]:
nba_teams = pd.DataFrame(teams.get_teams())
nba_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [3]:
team_ids = nba_teams['id'].unique()
team_ids

array([1610612737, 1610612738, 1610612739, 1610612740, 1610612741,
       1610612742, 1610612743, 1610612744, 1610612745, 1610612746,
       1610612747, 1610612748, 1610612749, 1610612750, 1610612751,
       1610612752, 1610612753, 1610612754, 1610612755, 1610612756,
       1610612757, 1610612758, 1610612759, 1610612760, 1610612761,
       1610612762, 1610612763, 1610612764, 1610612765, 1610612766])

# Getting all game records of season 20/21 up till now

In [10]:
start_time = time.time()

main_df = pd.DataFrame()
for team_id in team_ids:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable="2020-21", season_type_nullable="Regular Season")
    games = gamefinder.get_data_frames()[0]
    main_df = main_df.append(games)

main_df = main_df.sort_values('GAME_DATE',ascending=False)

end_time = time.time()
print('Execution time = %.6f seconds' % (end_time-start_time))

Execution time = 21.352790 seconds


In [7]:
main_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22020,1610612755,PHI,Philadelphia 76ers,0022001003,2021-05-07,PHI vs. NOP,W,240,109,...,0.818,10,41,51,27,5,6,15,19,2.0
0,22020,1610612757,POR,Portland Trail Blazers,0022001010,2021-05-07,POR vs. LAL,W,241,106,...,0.952,6,38,44,22,6,2,8,20,5.0
0,22020,1610612766,CHA,Charlotte Hornets,0022001004,2021-05-07,CHA vs. ORL,W,241,122,...,0.758,11,40,51,28,5,4,11,23,10.0
0,22020,1610612738,BOS,Boston Celtics,0022000160,2021-05-07,BOS @ CHI,L,239,99,...,0.783,11,36,47,19,5,8,13,15,-22.0
0,22020,1610612748,MIA,Miami Heat,0022001005,2021-05-07,MIA vs. MIN,W,240,121,...,0.741,8,29,37,27,9,2,14,22,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,22020,1610612766,CHA,Charlotte Hornets,0022000010,2020-12-23,CHA @ CLE,L,240,114,...,0.533,8,24,32,29,10,4,15,17,-7.0
66,22020,1610612747,LAL,Los Angeles Lakers,0022000002,2020-12-22,LAL vs. LAC,L,240,109,...,0.774,8,37,45,22,4,2,19,20,-7.0
66,22020,1610612751,BKN,Brooklyn Nets,0022000001,2020-12-22,BKN vs. GSW,W,242,125,...,0.813,13,44,57,24,11,7,20,22,26.0
66,22020,1610612744,GSW,Golden State Warriors,0022000001,2020-12-22,GSW @ BKN,L,240,99,...,0.652,13,34,47,26,6,6,18,24,-26.0


In [21]:
main_df.to_csv('nba_api_0705.csv') # create base database, subsequent days to be added on

# Get games for yesterday

In [22]:
yesterday = datetime.datetime.now() - datetime.timedelta(hours=36)  # based on US timezone
ytd_date = yesterday.date().strftime("%Y-%m-%d")   # convert to string format
ytd_date

'2021-05-06'

In [23]:
df = pd.DataFrame()
for team_id in team_ids:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    games = gamefinder.get_data_frames()[0]
    games = games[games['GAME_DATE'] == ytd_date] # the NBA season usually starts around october
    df = df.append(games)

df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22020,1610612737,ATL,Atlanta Hawks,0022001000,2021-05-06,ATL @ IND,L,239,126,...,0.833,10.0,24.0,34.0,27,8.0,6,12,17,-7.0
1,22020,1610612741,CHI,Chicago Bulls,0022000434,2021-05-06,CHI @ CHA,W,240,120,...,0.588,15.0,41.0,56.0,33,10.0,8,17,20,21.0
1,22020,1610612742,DAL,Dallas Mavericks,0022000998,2021-05-06,DAL vs. BKN,W,240,113,...,0.692,12.0,39.0,51.0,23,10.0,3,12,15,4.0
0,22020,1610612744,GSW,Golden State Warriors,0022001001,2021-05-06,GSW vs. OKC,W,238,118,...,0.833,7.0,42.0,49.0,36,4.0,1,13,19,21.0
0,22020,1610612746,LAC,LA Clippers,0022001002,2021-05-06,LAC vs. LAL,W,239,118,...,0.885,9.0,39.0,48.0,23,4.0,5,22,25,24.0
1,22020,1610612747,LAL,Los Angeles Lakers,0022001002,2021-05-06,LAL @ LAC,L,241,94,...,0.621,11.0,23.0,34.0,21,8.0,6,10,23,-24.0
0,22020,1610612751,BKN,Brooklyn Nets,0022000998,2021-05-06,BKN @ DAL,L,241,109,...,0.933,10.0,41.0,51.0,18,8.0,4,13,18,-4.0
0,22020,1610612754,IND,Indiana Pacers,0022001000,2021-05-06,IND vs. ATL,W,240,133,...,0.895,4.0,32.0,36.0,35,11.0,4,15,19,7.0
0,22020,1610612760,OKC,Oklahoma City Thunder,0022001001,2021-05-06,OKC @ GSW,L,241,97,...,0.800,5.0,34.0,39.0,21,9.0,1,9,19,-21.0
0,22020,1610612761,TOR,Toronto Raptors,0022000997,2021-05-06,TOR vs. WAS,L,265,129,...,0.778,18.0,32.0,50.0,23,9.0,5,12,30,-2.0
